# Colab Launcher – LoRA Training for FLAN-T5 on BioLaySumm

This notebook mirrors the Slurm script `scripts/slurm/train_flant5_base_lora.sbatch` and runs the repository code directly (no notebook-specific code).


In [ ]:
#@title 1) GPU and environment
nvidia-smi || true
pip install -q "transformers>=4.40" "datasets>=2.18" "evaluate>=0.4.2" peft rouge-score accelerate tensorboard


In [ ]:
#@title 2) Clone repo and cd
rm -rf PatternAnalysis-2025 || true
git clone https://github.com/0NATE4/PatternAnalysis-2025.git
cd PatternAnalysis-2025/recognition/layrad-flant5-lora-nchung
pwd
ls -la


In [ ]:
#@title 3) Optional: mount Google Drive for persistent checkpoints
try:
    from google.colab import drive
    drive.mount('/content/drive')
    print('Drive mounted')
    # optionally override output dir in YAML via sed below
except Exception as e:
    print('Drive not available:', e)


In [ ]:
#@title 4) Run LoRA training
python src/train.py configs/train_flant5_base_lora.yaml
